<a href="https://colab.research.google.com/github/nurulc/ATM/blob/master/tf_tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import drive

import altair as alt

import os

In [2]:
![[ -f housing-ma.csv.1 ]] || echo no

no


# Read MA Housing data

In [3]:

#drive.mount('content/gdrive')
# !head drive/My\ Drive/housing-ma.csv
#!head "/content/drive/My Drive/Colab Notebooks/Copy of tf-tutor.ipynb"

![[ -f housing-ma.csv ]] || wget https://raw.githubusercontent.com/nurulc/ATM/master/housing-ma.csv 

!head housing-ma.csv



CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98,24.00
0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14,21.60
0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03,34.70
0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94,33.40
0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33,36.20
0.02985,0.00,2.180,0,0.4580,6.4300,58.70,6.0622,3,222.0,18.70,394.12,5.21,28.70
0.08829,12.50,7.870,0,0.5240,6.0120,66.60,5.5605,5,311.0,15.20,395.60,12.43,22.90
0.14455,12.50,7.870,0,0.5240,6.1720,96.10,5.9505,5,311.0,15.20,396.90,19.15,27.10
0.21124,12.50,7.870,0,0.5240,5.6310,100.00,6.0821,5,311.0,15.20,386.63,29.93,16.50


In [4]:
!ls


drive  gdrive  housing-ma.csv  sample_data


# Read MA housing data into a pandas table

In [5]:
import pandas as pd
import io
data_pd = pd.read_csv(io.FileIO('housing-ma.csv'))
data_pd


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [6]:
dtype=np.double
def col_sort(t,col=0,descending=False):
    return t[t[:, col].sort(descending=descending)[1]]



# Normalize the data (function)

For each column of X(data)  subtract the mean of the column and scale by the range $$(highest - lowest)$$

This normalization maked the algoririthm more stable

> _Note:_ <span color="red">Do not mormalize the price column (MEDV)</span>

In [7]:
def norm(X):
    R=(np.amax(X,0)-np.amin(X,0)).reshape(-1,1)
    R[X.shape[1]-1][0] = 1.0
    M=np.mean(X,0).reshape(-1,1)
    M[X.shape[1]-1][0] = 0.0
    Mv = M.dot(np.ones((1,X.shape[0]),dtype=dtype)).transpose()
    Sc = R.dot(np.ones((1,X.shape[0]),dtype=dtype)).transpose()
    return [(X-Mv)/Sc,R,M]

# Setup MA housing data for regression

1. Convert pandas data to numpy
2. Place in X all the columns except the price
3. Add a '1's column to X _(this is to incorporate the intercept)_
4. Put into y the house prices where $y_i$ price corresponding to row $X_i$ 
5. Initialize $\phi$ with '1's


In [8]:


# !head sample_data/housing-ma.csv
ones = np.ones
cat = np.concatenate

Z = data_pd.to_numpy(dtype=np.double,copy=True)
m = Z.shape[0] # m = number of rows of data
n = Z.shape[1]-1 # n = number of columns excluding the house price
X = Z[:,0:n] # remove the house price (last column)

Ones = ones(m,dtype=np.double).reshape(-1,1)
X = cat((X,Ones),1) # add a 1 to each row (to compute the intercept)
y = Z[:,n].reshape(-1,1) # get the house price
phi = np.ones(n+1,dtype=np.double).reshape(-1,1) # initial value of phi
print("initial phi\n", phi)
print("input\n", X)

print("house price (first 10)\n",y[0:10])


initial phi
 [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
input
 [[6.3200e-03 1.8000e+01 2.3100e+00 ... 3.9690e+02 4.9800e+00 1.0000e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 3.9690e+02 9.1400e+00 1.0000e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 3.9283e+02 4.0300e+00 1.0000e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 3.9690e+02 5.6400e+00 1.0000e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 3.9345e+02 6.4800e+00 1.0000e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 3.9690e+02 7.8800e+00 1.0000e+00]]
house price (first 10)
 [[24. ]
 [21.6]
 [34.7]
 [33.4]
 [36.2]
 [28.7]
 [22.9]
 [27.1]
 [16.5]
 [18.9]]


# Numpy Multivariate regression

This method uses gradient descent to iteratively mimimize the cost function (MSE - mean standard error).

## Iterative computation of parameters _phi_ (multi variable linear regression)

$phi (\phi) $ are the regression parematers

*  find $\phi$ that minimizes cost
  *  *where:*    $Cost^2 = MSE = \frac{(X\phi -y)^T(X\phi -y)}{2m}$
   * $\phi$ is a minimum when $\frac{\partial{(Cost^2)}}{\partial\phi} = 0$
   * i.e. $\frac{\partial{(Cost^2)}}{\partial\phi} = \frac{X^T(X\phi - y)}{m} = 0$
   * by gradient descent we can compute successive values of $\phi_j$ 
       * $  \phi_j = \phi_{j-1} - \alpha \frac{\partial{(Cost^2)}}{\partial\phi}$
       * Resulting in the following:
          * $  \phi_j = \phi_{j-1} - \alpha X^T(X\phi_{j-1} - y)$
          * $\alpha$ is the learning rate is a small scalar such as 0.05

#### Where  
*X* = input data of [$m \times n$] matrix 

*  m = number of data rows, 
*  n = number of independent attributes of each data row

*y* = output (house price), vector of length $m$

_phi_ ($\phi$) = regression coefficients
and

_Where_: X$\phi$ (matrix multiply) = prediced house price

$Predicted_i = \sum_{j=1}^{n} X_{ij} * \phi_j$

In [9]:
import math
tr = np.transpose

def cost(X,phi,Y):
    m = X.shape[0];
    return math.sqrt((1/(2*m))*(tr(X@phi - Y)@(X@phi - Y)).item())

# Gradient decent for finding $\phi$ to minimise cost

In [10]:


def npSolve(X,y,iter=2,alpha=0.05):
    W, Range,Mean = norm(X) # normalize the data

    phi = ones((X.shape[1],1),dtype=np.double)
    cost_list = []
    Y = y
    m = X.shape[0]
    
    for i in range(iter):
        delta = alpha*(1/m)*(W.T)@(W@phi - Y) 
        phi = phi - delta
        cost_val = cost(W,phi,Y)
        cost_list.append(cost_val)
        
    intercept = np.zeros_like(Mean)
    intercept[-1][0] = (phi*Mean/Range).sum().item()
    phi = phi/Range - intercept
    return [phi,cost_list]    

# Iterative computation of parameters _phi_ (multi variable linear regression)

#### Where  
*X* = input data of [$m \times n$] matrix 

*  m = number of data rows, 
*  n = number of independent attributes of each data row

*y* = output (house price), vector of length $m$

_phi_ ($\phi$) = regression coefficients
and

_Where_: X$\phi$ (matrix multiply) = prediced house price

$Predicted_i = \sum_{j=1}^{n} X_{ij} * \phi_j$


        

In [11]:
%%time
phi,cost_list = npSolve(X,y,2000,0.5)
print(cost(X,phi,y))

3.3086923215668995
CPU times: user 49.7 ms, sys: 1.67 ms, total: 51.4 ms
Wall time: 57 ms


# Perform Multivariate regression (directly - no iteration)

We previously saw that for minimum $\phi$
* We have the following
* $\frac{X^T(X\phi - y)}{m} = 0$
  * multiplying bost sides by $m$
* $X^T(X\phi - y) = 0$
  * Multiplying by $X^T$ - _distributive property_
* $X^T(X\phi) -  X^Ty = 0$
  * Rearanging
* $(X^TX)\phi =  X^Ty$
----
   * Multiply both sides by the inverse of $X^TX$

* $(X^TX)^{-1} (X^TX)\phi = (X^TX)^{-1} X^Ty$
   * since  ${(X^TX)^{-1} (X^TX)} = I (identity)$
* we get the following: $I \phi = (X^TX)^{-1} X^Ty$
----
 **Therefore we get the very simple formula for $\phi$**
   * *$\phi = (X^TX)^{-1} X^Ty$*
----
&nbsp;

Note the following rules for matrix multiply
* $A(BC) = (AB)C$
  * and
* $A(B+C) = AB + BC$ - _distributive property_



In [12]:
%%time

inv = np.linalg.inv # invert a matrix

# See how simple the formulation can be using only linear algebra
def solve(X,y):
  return inv(X.T@X) @ (X.T@y)

phi2 = solve(X,y)  
print("cost=", cost(X,phi2,y),"diff = ", cost(X,phi,y)-cost(X,phi2,y)) # compare the cost of direct method and iterative method
#[phi,phi2, cost(X,phi2,y)-cost(X,phi,y), np.abs((phi-phi2)/phi*100)]

cost= 3.308687895656616 diff =  4.425910283600842e-06
CPU times: user 1.02 ms, sys: 626 µs, total: 1.64 ms
Wall time: 1.57 ms


# New Section